In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Carregar o dataset
df = pd.read_excel('ready_for_filtering.xlsx')

# Lista de possíveis valores para agile_methods e metrics_category
agile_methods_list = ['Scrum', 'Kanban', 'ScrumBan', 'XP', 'Safe', 'Lean']  # Adicione os métodos ágeis utilizados
metrics_category_list = ['Cronograma e progresso', 'Produto', 'Processo', 'Tecnologia', 'Pessoas', 'Cliente']

# Nova lista de features com colunas expandidas
new_agile_columns = [f'agile_methods_{method.lower()}' for method in agile_methods_list]
new_metrics_columns = [f'metrics_category_{category.lower().replace(" ", "_")}' for category in metrics_category_list]

# Atualizar a lista de features
features = ['role', 'years_exp', 'org_size', 
            'use_metrics_planning', 'use_metrics_review', 
            'use_metrics_weekly', 'use_metrics_daily', 
            'use_metrics_retro'] + new_agile_columns + new_metrics_columns
# Preencher valores nulos com string vazia
df[features] = df[features].fillna('')

# Converter as colunas binárias "Sim"/"Não" para 1/0
df[features] = df[features].replace({'Sim': 1, 'Não': 0})


# Gerar vetores TF-IDF apenas para as colunas não binárias (texto)
tfidf_matrices = {}
tfidf_vectorizers = {}

for feature in features:
    if df[feature].dtype == 'object':  # Aplicar TF-IDF apenas em colunas de texto
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrices[feature] = tfidf_vectorizer.fit_transform(df[feature])
        tfidf_vectorizers[feature] = tfidf_vectorizer  # Salvar o vectorizer usado

    else:
        # Para colunas binárias, convertemos diretamente em matriz numérica
        tfidf_matrices[feature] = df[[feature]].values

# Função para calcular a similaridade total ponderada considerando todas as features
def calculate_total_similarity(profile_index, feature_weights=None):
    total_similarity = np.zeros(df.shape[0])  # Inicializa similaridade total

    # Definir pesos padrão para cada feature se não forem fornecidos
    if feature_weights is None:
        feature_weights = {feature: 1.0 for feature in features}  # Pesos iguais

    # Loop sobre cada feature para calcular sua similaridade e aplicar o peso
    for feature in features:
        if df[feature].dtype == 'object':
            # Similaridade com TF-IDF para colunas textuais
            cosine_sim = cosine_similarity(tfidf_matrices[feature][profile_index], tfidf_matrices[feature]).flatten()
        else:
            # Similaridade para colunas binárias
            cosine_sim = cosine_similarity([tfidf_matrices[feature][profile_index]], tfidf_matrices[feature]).flatten()

            # Aplicar peso maior para colunas binárias com valor 1
            if df[feature].iloc[profile_index] == 1:
                feature_weights[feature] *= 2.0  # Aumenta o peso se for 1 (ou "Sim")
        
        # Amplificar o impacto dos pesos para as features binárias e métricas importantes
        total_similarity += cosine_sim * feature_weights[feature]

    return total_similarity  # Retornar a similaridade sem normalizar novamente

# Função para recomendar métricas com base em perfis similares
def recommend_metrics(profile_index, top_n=5, feature_weights=None):
    # Calcular a similaridade total ponderada
    total_similarity = calculate_total_similarity(profile_index, feature_weights)

    # Verificar a distribuição da similaridade total
    print(f"Distribuição de similaridade: min={total_similarity.min()}, max={total_similarity.max()}, mean={total_similarity.mean()}")

    # Obter os índices dos perfis mais similares (excluindo o próprio perfil)
    similar_indices = total_similarity.argsort()[-(top_n+1):-1][::-1]

    # Obter as similaridades dos perfis selecionados
    similar_profiles = df.iloc[similar_indices]
    similar_affinity = total_similarity[similar_indices]

    # Exibir as afinidades diretamente sem normalização
    recommended_metrics = []
    
    # Loop para recomendar as métricas dos perfis mais similares
    for idx, metrics in enumerate(similar_profiles['sanitized_metrics']):
        if metrics not in [rec[0] for rec in recommended_metrics]:  # Verifica se a métrica já foi recomendada
            # todo debuggar isso
            affinity = similar_affinity[idx]  # Usar similaridade absoluta, sem normalização
            recommended_metrics.append((metrics, affinity, similar_indices[idx]))
        
        # Parar após encontrar 5 métricas diferentes
        if len(recommended_metrics) == top_n:
            break

    return recommended_metrics

# Definir pesos ajustados para amplificar o impacto das colunas binárias e importantes
feature_weights = {
    'role': 0.5,  # Peso menor para 'role'
    'years_exp': 0.5,
    'org_size': 0.5,
    'use_metrics_planning': 3.5,  # Amplificação do peso para estas colunas
    'use_metrics_review': 3.5,
    'use_metrics_weekly': 3.5,
    'use_metrics_daily': 3.5,
    'use_metrics_retro': 3.5,
    'agile_methods_scrum': 5.0,  # Peso ainda maior para métodos ágeis
    'agile_methods_kanban': 5.0,
    'agile_methods_scrumban': 5.0,
    'agile_methods_lean': 5.0,
    'agile_methods_xp': 5.0,
    'agile_methods_safe': 5.0,
    'metrics_category_cronograma_e_progresso': 3.5,  # Amplificação do peso para categorias de métricas
    'metrics_category_produto': 3.5,
    'metrics_category_processo': 3.5,
    'metrics_category_tecnologia': 3.5,
    'metrics_category_pessoas': 3.5,
    'metrics_category_cliente': 3.5
}

# Exemplo de uso: recomende métricas para o perfil com id_integer 8
profile_id = 65
profile_index = df[df['id_integer'] == profile_id].index[0]

# Recomendar métricas para o perfil
recommended_metrics = recommend_metrics(profile_index, top_n=5, feature_weights=feature_weights)

# Exibir as métricas recomendadas, suas porcentagens de afinidade e o índice do perfil similar
for metric, affinity, similar_index in recommended_metrics:
    print(f"Métrica: {metric}, Similaridade Absoluta: {affinity:.2f}, Índice do Perfil Similar: {similar_index}")

import joblib

# Definir o caminho para salvar o modelo
model_path = 'collaborative_filtering.joblib'

# Criar um dicionário com todos os elementos necessários para o modelo
model_data = {
    'tfidf_matrices': tfidf_matrices,  # Inclui suas matrizes TF-IDF
    'feature_weights': feature_weights,
    'df': df,
    'features': features,
    'tfidf_vectorizers': tfidf_vectorizers  # Inclui os vectorizers

}

# Salvar o dicionário com os dados do modelo usando joblib
joblib.dump(model_data, model_path)

print(f"Modelo salvo em: {model_path}")

Distribuição de similaridade: min=8.054756544279513, max=73.5, mean=47.99915894639142
Métrica: Throughput, Similaridade Absoluta: 73.50, Índice do Perfil Similar: 160
Métrica: Team total available hours, Similaridade Absoluta: 73.50, Índice do Perfil Similar: 161
Métrica: CAC, Similaridade Absoluta: 73.11, Índice do Perfil Similar: 332
Métrica: Lead time, Similaridade Absoluta: 73.00, Índice do Perfil Similar: 95
Modelo salvo em: collaborative_filtering.joblib


/var/folders/5j/0g6pzkwj7_g1p9xwqx5jfgq80000gn/T/ipykernel_8177/3651758904.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[features] = df[features].replace({'Sim': 1, 'Não': 0})
